Overhead

In [3]:
#Loosely translated version of the R code, rework to make runable
packages_vector = ["tidyverse", "arrow", "stringdist"]

for i in packages_vector:
    try:
        return __import__(i)
    except ImportError:
        return None
    
#!pip list

SyntaxError: 'return' outside function (Temp/ipykernel_17304/2290899003.py, line 5)

In [16]:
pip install pyarrow

In [1]:
#Dependencies loading- clean up after all are identified
import pandas as pd
import numpy as np
from pyarrow import feather

In [2]:
#Remote I/O
#interim_dir = "../../data/interim/"
#lasso_lookup_dir = "../../../mtc-Lasso/mtc_data/lookups/"

interim_dir = "E:/Box/Modeling and Surveys/Development/Travel Model Two Development/Travel Model Two Network Rebuild/travel-model-two-networks/data/interim/"
lasso_lookup_dir = "C:/Users/natchison/Documents/GitHub/Lasso/mtc_data/lookups/"

links_file_name = interim_dir + "step5_tidy_roadway/link.feather"

conflation_file_name = interim_dir + 'conflation_result.csv'

output_legacy_tm2_file_name = lasso_lookup_dir + "legacy_tm2_attributes.csv"
output_tam_tm2_file_name = lasso_lookup_dir + "tam_tm2_attributes.csv"
output_pems_file_name = lasso_lookup_dir + "pems_attributes.csv"
output_sfcta_file_name = lasso_lookup_dir + "sfcta_attributes.csv"
output_tomtom_file_name = lasso_lookup_dir + "tomtom_attributes.csv"
output_osm_file_name = lasso_lookup_dir + "osm_lanes_attributes.csv"

In [3]:
#Parameters
frc_labels = ["Not Applicable", "Motorway, Freeway, or Other Major Road", 
              "Major Road Less Important than a Motorway", "Other Major Road",
              "Secondary Road", "Local Connecting Road",
              "Local Road of High Importance", "Local Road",
              "Local Road of Minor Importance", "Other Road"]

ft_labels = ["Connector", "Freeway to Freeway",
             "Freeway", "Expressway",
             "Collector", "Ramp",
             "Special Facility", "Major Arterial"]

frc_dict_df = pd.DataFrame({'code' : list(range(-1,9)), 'label' : frc_labels}) #Drop index if possible
ft_dict_df = pd.DataFrame({'code' : list(range(0,8)), 'label' : ft_labels})

In [4]:
#Data Reads
links_df = pd.read_feather(links_file_name)
conflation_df = pd.read_csv(conflation_file_name) #may need to change col type to character

C:\Users\natchison\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
#Reductions osm_df CONFIRMED
osm_df = links_df[links_df.drive_access == 1]
osm_df = osm_df[['shstReferenceId','lanes']]
osm_df = osm_df.rename(columns={'lanes':'lanes_str'})

osm_df['min_lanes'] = np.nan
osm_df.loc[osm_df['lanes_str'].str.contains('12'), 'min_lanes'] = 12
osm_df.loc[osm_df['lanes_str'].str.contains('11'), 'min_lanes'] = 11
osm_df.loc[osm_df['lanes_str'].str.contains('10'), 'min_lanes'] = 10
osm_df.loc[osm_df['lanes_str'].str.contains('9'), 'min_lanes'] = 9
osm_df.loc[osm_df['lanes_str'].str.contains('8'), 'min_lanes'] = 8
osm_df.loc[osm_df['lanes_str'].str.contains('7'), 'min_lanes'] = 7
osm_df.loc[osm_df['lanes_str'].str.contains('6'), 'min_lanes'] = 6
osm_df.loc[osm_df['lanes_str'].str.contains('5'), 'min_lanes'] = 5
osm_df.loc[osm_df['lanes_str'].str.contains('4'), 'min_lanes'] = 4
osm_df.loc[osm_df['lanes_str'].str.contains('3'), 'min_lanes'] = 3
osm_df.loc[osm_df['lanes_str'].str.contains('2'), 'min_lanes'] = 2
osm_df.loc[osm_df['lanes_str'].str.contains('1'), 'min_lanes'] = 1

osm_df['max_lanes'] = osm_df['min_lanes']
osm_df.loc[osm_df['lanes_str'].str.contains('2'), 'max_lanes'] = 2
osm_df.loc[osm_df['lanes_str'].str.contains('3'), 'max_lanes'] = 3
osm_df.loc[osm_df['lanes_str'].str.contains('4'), 'max_lanes'] = 4
osm_df.loc[osm_df['lanes_str'].str.contains('5'), 'max_lanes'] = 5
osm_df.loc[osm_df['lanes_str'].str.contains('6'), 'max_lanes'] = 6
osm_df.loc[osm_df['lanes_str'].str.contains('7'), 'max_lanes'] = 7
osm_df.loc[osm_df['lanes_str'].str.contains('8'), 'max_lanes'] = 8
osm_df.loc[osm_df['lanes_str'].str.contains('9'), 'max_lanes'] = 9
osm_df.loc[osm_df['lanes_str'].str.contains('10'), 'max_lanes'] = 10
osm_df.loc[osm_df['lanes_str'].str.contains('11'), 'max_lanes'] = 11
osm_df.loc[osm_df['lanes_str'].str.contains('12'), 'max_lanes'] = 12

osm_df = osm_df.dropna()
osm_df = osm_df[['shstReferenceId', 'min_lanes', 'max_lanes']]
#osm_df.head(20)

In [213]:
#Reductions osm_names_df - Semi Confirmed (Ask Flavia about the fact that the R Script is missing some simple names)
osm_names_df = links_df[links_df.drive_access == 1]
osm_names_df = osm_names_df[['shstReferenceId','name']]

osm_names_df['name'] = osm_names_df['name'].str.replace("'","").str.replace("\\[","").str.replace("\\]","")
names_split = osm_names_df['name'].str.split(',', n=5, expand=True)
names_split = names_split.iloc[: , :-1]
osm_names_df = osm_names_df[['shstReferenceId']]
osm_names_df = pd.concat([osm_names_df, names_split], axis=1)
osm_names_df = osm_names_df.melt(id_vars=['shstReferenceId'],value_vars=[*range(0,5)],
                                 var_name='index',value_name='name')

osm_names_df = osm_names_df[['shstReferenceId','name']]
osm_names_df['name'] = osm_names_df['name'].str.strip()
osm_names_df = osm_names_df.dropna()
osm_names_df = osm_names_df[osm_names_df.name != 'nan']
osm_names_df = osm_names_df[osm_names_df.name != '']

osm_names_df = osm_names_df.drop_duplicates()
osm_names_df['index'] = osm_names_df.groupby(['shstReferenceId']).shstReferenceId.transform('cumcount')
osm_names_df['index'] = osm_names_df['index'] + 1

osm_names_df = osm_names_df.pivot(index='shstReferenceId', columns='index', values='name').add_prefix("name_")
osm_names_df['name'] = osm_names_df['name_1']
osm_names_df.loc[~osm_names_df['name_2'].isnull(), 'name'] = osm_names_df['name'] + '/' + osm_names_df['name_2']
osm_names_df.loc[~osm_names_df['name_3'].isnull(), 'name'] = osm_names_df['name'] + '/' + osm_names_df['name_3']

remove_cols = [' Avenue',' Street',' Drive',' Boulevard',' Way']
osm_names_df['simple_name'] = osm_names_df['name'].str.replace('|'.join(remove_cols),'')
osm_names_df = osm_names_df.reset_index()
osm_names_df = osm_names_df[['shstReferenceId','name','simple_name']]
osm_names_df['simple_name'] = osm_names_df['simple_name'].str.strip()

#osm_names_df.head(30)

C:\Users\NATCHI~1\AppData\Local\Temp/ipykernel_20348/2805073947.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  osm_names_df['name'] = osm_names_df['name'].str.replace("'","").str.replace("\\[","").str.replace("\\]","")
C:\Users\NATCHI~1\AppData\Local\Temp/ipykernel_20348/2805073947.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  osm_names_df['simple_name'] = osm_names_df['name'].str.replace('|'.join(remove_cols),'')


index,shstReferenceId,name,simple_name
0,00000b467d2e08f9abf13eeafee3ed46,Yolanda Circle,Yolanda Circle
1,000018062272093dbaa5d49303062454,Molino Avenue,Molino
2,000018a23d1330dcfbae79f44e9fca1a,Mina Way,Mina
3,00003727c3137224cb1a5c95df9fc8a5,Merlot Way,Merlot
4,00004e60296f9a9faa45248283397c28,Saratoga-Sunnyvale Road,Saratoga-Sunnyvale Road
5,0000611f41ef8330ddc30df951166a22,Stone Valley Road,Stone Valley Road
6,00008f3db470b7993ed7efe3fdbbe371,29th Avenue,29th
7,0000b310722686c9f3f8c92ea0cd4536,Hulme Court,Hulme Court
8,0000b55e3cdf5c413d2be14640d65e17,Tahoe Drive,Tahoe
9,0000ef92881341b20a23b16439cd8cc6,Bruns Road,Bruns Road


In [28]:
#Reductions - legacy_df CONFIRMED
legacy_df = conflation_df.dropna(subset=['TM2_A'])
legacy_df = legacy_df[['shstReferenceId','TM2_A','TM2_B','TM2_FT',
                       'TM2_LANES', 'TM2_ASSIGNABLE']]

legacy_df = legacy_df.rename(columns={'shstReferenceId':'shstReferenceId',
                                      'TM2_A':'A_node',
                                      'TM2_B':'B_node',
                                      'TM2_FT':'code', 
                                      'TM2_LANES':'lanes',
                                      'TM2_ASSIGNABLE':'assignable'})
convert_dict = {'A_node': int,
                'B_node': int,
                'code': int,
                'lanes': int,
                'assignable': int}
legacy_df = legacy_df.astype(convert_dict)
legacy_df = legacy_df.merge(ft_dict_df, on='code', how='left')
legacy_df = legacy_df.rename(columns={'label':'ft'})
legacy_df = legacy_df.drop(['code'], axis=1)
#legacy_df

,shstReferenceId,A_node,B_node,lanes,assignable,ft
0,00000b467d2e08f9abf13eeafee3ed46,3027596,3028181,1,1,Collector
1,000018062272093dbaa5d49303062454,5005807,5010629,1,1,Collector
2,000018a23d1330dcfbae79f44e9fca1a,2002804,2113098,1,1,Major Arterial
3,00004e60296f9a9faa45248283397c28,2042055,2049818,2,1,Major Arterial
4,00008f3db470b7993ed7efe3fdbbe371,1005731,1004692,1,1,Collector
...,...,...,...,...,...,...
757718,fffee6ff45b20090922f0228982be4f7,1501909,1514389,1,1,Expressway
757719,ffff07397a2a2ec5c0c422ca4e00593e,2021199,2023234,1,1,Collector
757720,ffff0b93dcc6e8e41cc44a399af9755f,2552239,2551260,1,1,Collector
757721,ffff0f2818ca9ce39f2ea8f5b8450bb1,4506195,4517894,1,1,Collector


In [34]:
#Reductions - tam_df CONFIRMED
tam_df = conflation_df.dropna(subset=['TM2Marin_A'])
tam_df = tam_df[['shstReferenceId','TM2Marin_A','TM2Marin_B','TM2Marin_FT',
                       'TM2Marin_LANES', 'TM2Marin_ASSIGNABLE']]

tam_df = tam_df.rename(columns={'shstReferenceId':'shstReferenceId',
                                      'TM2Marin_A':'A_node',
                                      'TM2Marin_B':'B_node',
                                      'TM2Marin_FT':'code', 
                                      'TM2Marin_LANES':'lanes',
                                      'TM2Marin_ASSIGNABLE':'assignable'})
convert_dict = {'A_node': int,
                'B_node': int,
                'code': int,
                'lanes': int,
                'assignable': int}
tam_df = tam_df.astype(convert_dict)
tam_df = tam_df.merge(ft_dict_df, on='code', how='left')
tam_df = tam_df.rename(columns={'label':'ft'})
tam_df = tam_df.drop(['code'], axis=1)
#tam_df

In [36]:
#Add and filter pems id CONFIRMED
pems_cols = [col for col in conflation_df.columns if 'pems' in col] 
pems_df = conflation_df[['shstReferenceId'] + ['PEMSID'] + pems_cols]

pems_df['pems_ft'] = np.nan
pems_df.loc[~pems_df['pems_lanes_FR'].isnull(), 'pems_ft'] = 'Ramp'
pems_df.loc[~pems_df['pems_lanes_OR'].isnull(), 'pems_ft'] = 'Ramp'
pems_df.loc[~pems_df['pems_lanes_FF'].isnull(), 'pems_ft'] = 'Freeway to Freeway'
pems_df.loc[~pems_df['pems_lanes_ML'].isnull(), 'pems_ft'] = 'Freeway'
pems_df.loc[~pems_df['pems_lanes_HV'].isnull(), 'pems_ft'] = 'Freeway'

pems_df['pems_lanes'] = np.nan
pems_df.loc[~pems_df['pems_lanes_FR'].isnull(), 'pems_lanes'] = pems_df['pems_lanes_FR']
pems_df.loc[~pems_df['pems_lanes_OR'].isnull(), 'pems_lanes'] = pems_df['pems_lanes_OR']
pems_df.loc[~pems_df['pems_lanes_FF'].isnull(), 'pems_lanes'] = pems_df['pems_lanes_FF']
pems_df.loc[~pems_df['pems_lanes_ML'].isnull(), 'pems_lanes'] = pems_df['pems_lanes_ML']
pems_df.loc[~pems_df['pems_lanes_HV'].isnull(), 'pems_lanes'] = pems_df['pems_lanes_HV'] + pems_df['pems_lanes']

pems_df = pems_df.drop(pems_cols, axis=1)
pems_df = pems_df.rename(columns={'pems_ft':'ft', 'pems_lanes':'lanes'})
pems_df = pems_df.dropna()
pems_df['lanes'] = pems_df['lanes'].astype('int')

#pems_df

C:\Users\NATCHI~1\AppData\Local\Temp/ipykernel_20348/294684268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pems_df['pems_ft'] = np.nan
C:\Users\NATCHI~1\AppData\Local\Temp/ipykernel_20348/294684268.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pems_df['pems_lanes'] = np.nan


In [71]:
#Tom_df CONFIRMED
tom_df = conflation_df.dropna(subset=['tomtom_unique_id'])
tom_df = tom_df[['shstReferenceId','tomtom_unique_id','tomtom_FRC','tomtom_lanes',
                       'tomtom_shieldnum', 'tomtom_rtedir']]

tom_df = tom_df.rename(columns={'tomtom_unique_id':'tom_id',
                                      'tomtom_FRC':'code',
                                      'tomtom_lanes':'lanes'})
convert_dict = {'code': int,
                'lanes': int}
tom_df = tom_df.astype(convert_dict)
tom_df = tom_df.merge(frc_dict_df, on='code', how='left')
tom_df = tom_df.rename(columns={'label':'ft'})
tom_df = tom_df.drop(['code'], axis=1)

tom_df.loc[tom_df['tomtom_shieldnum']==' ', 'tomtom_shieldnum'] = 'NA'
tom_df.loc[tom_df['tomtom_rtedir']==' ', 'tomtom_rtedir'] = 'NA'

tom_df['name'] = tom_df['tomtom_shieldnum'] + [' '] + tom_df['tomtom_rtedir']
tom_df = tom_df.drop(['tomtom_shieldnum', 'tomtom_rtedir'], axis=1)
#tom_df

In [217]:
#SFCTA DF NOT YET DONE
sfcta_df = conflation_df.dropna(subset=['sfcta_A'])
sfcta_df = sfcta_df[['shstReferenceId','sfcta_A','sfcta_B','sfcta_LANE_AM',
                       'sfcta_LANE_OP', 'sfcta_LANE_PM', 'sfcta_STREETNAME']]

sfcta_df = sfcta_df.rename(columns={'sfcta_A':'A_node',
                                    'sfcta_B':'B_node',
                                    'sfcta_LANE_AM':'lanes_am',
                                    'sfcta_LANE_OP':'lanes_md',
                                    'sfcta_LANE_PM':'lanes_pm',
                                    'sfcta_STREETNAME':'name'})
convert_dict = {'A_node': int,
                'B_node': int,
                'lanes_am': int,
                'lanes_md': int,
                'lanes_pm': int}
sfcta_df = sfcta_df.astype(convert_dict)

sfcta_df['lanes_min'] = sfcta_df[['lanes_am','lanes_md','lanes_pm']].min(axis=1)
sfcta_df['lanes_max'] = sfcta_df[['lanes_am','lanes_md','lanes_pm']].max(axis=1)
sfcta_df = sfcta_df.merge(osm_names_df[osm_names_df], on='shstReferenceId', how='left')
sfcta_df['osm_sfcta_names_dist'] = 100

sfcta_df

,shstReferenceId,A_node,B_node,lanes_am,lanes_md,lanes_pm,name_x,lanes_min,lanes_max,name_y,simple_name,osm_sfcta_names_dist
0,00008f3db470b7993ed7efe3fdbbe371,27834,27832,1,1,1,29TH,1,1,29th Avenue,29th,100
1,000135f06e99cfa717aac0daeb35ca1e,20766,20764,1,1,1,ELMIRA,1,1,Elmira Street,Elmira,100
2,00040e43e020b2d01c78443b2dd421ac,27153,27156,2,2,3,LINCOLN,2,3,Lincoln Way,Lincoln,100
3,000436c66cd025d0882221e1802db32d,21255,21258,1,1,1,HIGHLAND,1,1,Patton Street,Patton,100
4,000c52714d350c96e2909c3daa4b9013,21897,33362,2,2,2,CESAR CHAVEZ,2,2,Cesar Chavez Street,Cesar Chavez,100
...,...,...,...,...,...,...,...,...,...,...,...,...
34901,fff4922b58f884f314af40b56cf6bb6c,21272,21282,1,1,1,COSO,1,1,Stoneman Street,Stoneman,100
34902,fff7462ba361ef9d7712033765f84757,21237,21239,1,1,1,BENTON,1,1,Benton Avenue,Benton,100
34903,fff9839602461c6f214ce259e2dfe121,24618,24605,1,1,1,2ND,1,1,2nd Street,2nd,100
34904,fffcb6120687d1b2609505da4e1b255f,23897,23916,3,2,3,6TH,2,3,6th Street,6th,100


In [214]:
#Table comparision
df = osm_names_df
df_r = pd.read_csv("E:\\Local Data\\osm_names_df.csv")

df_r = df_r.iloc[: , 1:]

t = df.merge(df_r, how ='left',
                     on=list(df), indicator=True)
t[t['_merge'] == 'both'].shape[0]
t[t['_merge'] == 'left_only'].shape[0]

1863